İlk olarak, model oluştururken kullanılacak tüm kütüphaneleri tek bir Jupyter not defteri hücresinde içe aktaralım:

In [1]:
import pandas as pd
import numpy as np
import pickle 

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [2]:
# Tam Veri Kümesi (Full Dataset)
full_set = pd.read_csv('./data/full_dataset.csv')
full_set.head(5)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
full_set.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

Veri kümesindeki değişkenlerin veri tiplerine (data types) bakalım:

In [4]:
full_set.dtypes

# Gender                object
# Married               object
# Dependents            object
# Education             object
# Self_Employed         object
# ApplicantIncome        int64
# CoapplicantIncome    float64
# LoanAmount           float64
# Loan_Amount_Term     float64
# Credit_History       float64
# Property_Area         object
# Loan_Status           object
# dtype: object

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

Veri kümesi hem nicel (nümerik, sayısal) hem de kategorik (nitel) değişkenlerden oluşmaktadır.

Tam veri kümesinin (`full_dataset.csv`) şekline bakalım:

In [5]:
print(full_set.shape)
# (614, 13)

(614, 13)


Etiketli (labelled) veri kümesinde 614 gözlem (observation) ve 13 sütun vardır. Burada `Loan_ID` sütunu modelimiz için herhangi bir bilgi taşımamaktadır, bu  nedenle, bu sütunu elimine edebiliriz. `Loan_Status` değişkeni bağımlı değişkendir. Bu bir tarihi veridir (historical data). Geçmişte, müşterilerin kredi başvurusunun onaylanıp onaylanmadığı bilgisini taşımaktadır. Müşterilerin (yani gözlemlerin) bilgisi ise `Gender`, `Married`, `Dependents`, `Education`, `Self_Emplyed`, `ApplicantIncome`, `CoapplicantIncome`, `LoanAmount`, `Loan_Amount_Term`, `Credit_History` ve `Property_Area` değişkenleri ile verilmektedir.

İlk olarak bu tam veri kümesinden, analizimizde kullanmayacağımız, `Loan_ID` sütununu düşürelim:  

In [6]:
full_set.drop(['Loan_ID'], axis=1, inplace=True)
full_set.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


Bağımlı değişken (dependent variable) olan `Loan_Status` değişkeni Y (Yes) ve N (No) olmak üzere iki kategoriden oluşmaktadır. Bu değişkeni nümerikleştirelim. `Y` kategorisi için `1.0`; `N` kategorisi için `0.0` eşleşmesi gerçekleştirelim:

In [7]:
full_set['Loan_Status'] = full_set['Loan_Status'].map({'N':0,'Y':1})
full_set.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1


Şimdi, elimizdeki tam veri kümesinde kayıp gözlem olup olmadığını kontrol edelim:

In [8]:
full_set.isnull().sum()

# Gender               13
# Married               3
# Dependents           15
# Education             0
# Self_Employed        32
# ApplicantIncome       0
# CoapplicantIncome     0
# LoanAmount           22
# Loan_Amount_Term     14
# Credit_History       50
# Property_Area         0
# Loan_Status           0
# dtype: int64

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

Görüldüğü üzere bir kaç değişkende kayıp gözlem (missing value) vardır. Ancak, kayıp gözlem imputasyonunu (missing value imputation) tam veri kümesine uygulamak veri sızıntısına (data leakage) neden olacaktır. Bu nedenle, öncelikle tam veri kümesini, $\%80 - \%20$ oranı kullanılarak iki kısıma ayıralım. $\%80$'lik kısıma eğitim veri kümesi (training dataset), $\%20$'lik kısıma test veri kümesi (testing dataset) olacaktır. İşte, eğitim veri kümesi parametreleri kullanılarak, test veri kümesi üzerinde veri imputasyonu gerçekleştirebilir.

Veri kümesine ayırma işlemi Scikit-Learn kütüphanesindeki `train_test_split` fonksiyonu kullanılarak yapılır. Ancak, bu fonksiyon, bağımlı değişken(ler) ile bağımsız değişken(ler)i ayrı ayrı kabul etmektedir. Bu nedenle, bağımlı değişken olan `Loan_Status` değişkeni başka bir değişkene atılalım:

In [9]:
X_full = full_set.drop(["Loan_Status"], axis=1)
y_full = full_set["Loan_Status"]

In [10]:
X_full

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban
...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural
610,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural
611,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban
612,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban


Burada, `X_full` bağımsız değişkenlerin (özniteliklerin - features) olduğu bir matristir.

Artık, tam veri kümemizi parçalayabiliriz.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size  = 0.2, random_state=42)
print(f"Eğitim Kümesinin şekli: {X_train.shape}, \nTest Kümesinin Şekli: {X_test.shape}")

# Eğitim Kümesinin şekli: (491, 11), 
# Test Kümesinin Şekli: (123, 11)

Eğitim Kümesinin şekli: (491, 11), 
Test Kümesinin Şekli: (123, 11)


Kolaylıkla anlaşılabileceği üzere, eğitim veri kümesinde 491 müşteriye, test veri kümesinde ise 123 müşteriye ait bilgi bulunmaktadır. Bağımsız değişkenlerin sayısı ise 11'dir.

Burada eğitim veri kümesi çok büyük olmadığı için model doğrulama (model validation), çapraz doğrulama (cross validation) kullanılarak elde edilecektir. Çapraz doğrulama gerçekleştirilirken de, kullanacağımız makine öğrenmesi modelinin hiperparametrelerine Izgara Arama (Grid Search) kullanılarak ince ayar (fine tuning) verilecektir.

Ancak, veri kümesinde hem sayısal (nümerik) hem de kategorik (categorical) değişkenler vardır. Buna ek olarak, eğitim kümesinde kayıp gözlemler de mevcuttur. Bu farklı değişkenlere ayrı ayrı veri önişleme (data pre-processing) gerçekleştirilecektir. Örneğin, nümerik değişkenler ölçeklendirilecektir (scaling) ve kategorik değişkenler bire-bir-kodlama (one-hot-encoding) kullanılarak nümerikleştirilecektir.

Fakat, herhangi bir veri ön-işleme gerçekleştirirken, veri sızıntısını (data leakage) engellemek için , bu ön-işleme adımını, çapraz doğrulamanın her parçasında ayrı ayrı tekrarlamanız gerekmektedir. Bu adımlar gerek veri ölçekleme (data scaling) olabilir, gerekse sentetik veri oluşturma (Synthetic Data Generation - SMOTE, ADASYN, Oversampling, Undersampling) olabilir, veya kayıp değer tamamlama (missing value imputation) ya da kategorik değişken kodlama (categorical variable encoding) olabilir. 

Verideki nümerik (sayısal) ve kategorik değişkenleri ayırdıktan sonra, bu değişkenlerin türüne göre farklı veri önişleme adımlarını Scikit-Learn kütüphanesindeki `Pipeline` fonksiyonunu kullanarak bir iletim hattı üzerine oturtabilir ve daha sonra gene Scikit-Learn kütüphanesinde bulunan `ColumnTransformer` fonksiyonu yardımı ile bu dönüşleri ayrı ayrı uygulayabilirsiniz.

Daha sonra başka bir Pipeline (İletim Hattı) oluşturarak, önce bir önişleme gerçekleştirir ve daha sonra `GridSearchCV` fonksiyonu ile istediğiniz makine öğrenmesi modeli için çapraz doğrulama ile Izgara Arama (Grid Search) kullanarak hiperparametre araştırması gerçekleyebilirsiniz. İşte, arka planda k-parça çapraz doğrulama (k-fold cross validation) yapılırken, nümerik ve kategorik değişkenler için ayrı ayrı tanımladığınız tüm önişleme adımlarının parametreleri k-1 parçadaki verilerden elde edilecek ve daha sonra bu parametreler k-ıncı parçada uygulanacaktır ve bu, k defa aynı şekilde tekrarlanacaktır.

Öncelikle veri kümemizdeki nümerik ve kategorik değişkenlerin isimlerini otomatik olarak çekelim:

In [12]:
num_features = full_set.drop(['Loan_Status'], axis = 1).select_dtypes(include = 'number').columns
num_features
# Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
#        'Loan_Amount_Term', 'Credit_History'],
#       dtype='object')

Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History'],
      dtype='object')

`ApplicantIncome`, `CoapplicantIncome`, `LoanAmount`, `Loan_Amount_Term`, ve `Credit_History` değişkenleri nümerik veri yapısına sahiptir.

In [13]:
cat_features = full_set.select_dtypes(include = 'object').columns
cat_features
# Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
#        'Property_Area'],
#       dtype='object')

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'Property_Area'],
      dtype='object')

Benzer şekilde, `Gender`, `Married`, `Dependents`, `Education`, `Self_Employed`, ve `Property_Area` değişkenleri kategorik veri yapısına sahiptir.

Yukarıda da bahsedildiği gibi, nümerik ve kategorik değişkenlere ayrı ayrı veri ön-işleme adımları gerçekleştirilecektir. Nümerik değişkenler aynı birime sahip olmaları için ölçeklenecektir ve varsa kayıp gözlemler ortalam ile impute edilecektir. Benzer şekilde, kategorik değişkenler, makine öğrenmesi modelinin anlayabileceği şekilde bire-bir-kodlama (one-hot-encoding) kullanılarak nümerikleştirilecektir ve kayıp gözlem varsa sabit bir değer ile (örneğin, `missing` yani bilinmiyor bilgisi) ile impute edilecektir. Bu iki farklı değişken türü için gerçekleştirilecek işlemler, iki farklı iletim hattı ile gerçekleştirilecektir.

In [14]:
# Sayısal değişkenlere uygulanacak veri ön-işleme adımları için iletim hattı (pipeline)

num_transformer = Pipeline(steps = [('Imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
                                    ('MinMaxScaler', MinMaxScaler())])
# Pipeline(steps=[('Imputer', SimpleImputer()), ('MinMaxScaler', MinMaxScaler())])

# Kategorik değişkenlere uygulanacak veri ön-işleme adımları için iletim hattı (pipeline)

cat_transformer = Pipeline(steps = [('Imputer', SimpleImputer(strategy = 'constant', fill_value = 'missing')),
                                    ('OneHotEncoder', OneHotEncoder(categories = 'auto', drop=None, handle_unknown = 'ignore'))])

# Pipeline(steps=[('Imputer',
#                  SimpleImputer(fill_value='missing', strategy='constant')),
#                 ('OneHotEncoder',
#                  OneHotEncoder(drop='None', handle_unknown='ignore'))])

İletim hatlarını tanımladıktan sonra artık sütun dönüşümlerine (column transformers) geçebiliriz. 

In [15]:
preprocessor = ColumnTransformer(transformers = [('num', num_transformer, num_features), 
                                                 ('cat', cat_transformer, cat_features)],
                                 remainder = 'drop',
                                 n_jobs = -1,
                                 verbose = False)
# ColumnTransformer(n_jobs=-1,
#                   transformers=[('num',
#                                  Pipeline(steps=[('Imputer', SimpleImputer()),
#                                                  ('MinMaxScaler',
#                                                   MinMaxScaler())]),
#                                  Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
#        'Loan_Amount_Term', 'Credit_History'],
#       dtype='object')),
#                                 ('cat',
#                                  Pipeline(steps=[('Imputer',
#                                                   SimpleImputer(fill_value='missing',
#                                                                 strategy='constant')),
#                                                  ('OneHotEncoder',
#                                                   OneHotEncoder(drop='None',
#                                                                 handle_unknown='ignore'))]),
#                                  Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
#        'Property_Area'],
#       dtype='object'))])

Artık veri ön-işleme hattımız hazır!

Şimdi, bu veri önişleme adımı ile oluşturacağımız Rastgele Ormanlar (random forest) algoritmasını bir araya getirebilir, kolaylıkla çapraz doğrulama uygulayabiliriz. Böylelikle, yukarıda bahsedilen veri sızıntısından da kaçınırız. 

In [16]:
pipe = Pipeline(steps = [('preprocess', preprocessor),
                         ('RF_model', RandomForestClassifier(class_weight = "balanced", n_jobs=-1))],
                verbose=False)         

Daha sonra, ızgara arama gerçekleştirirken kullanacağımız parametreler için olası değerleri içerek bir sözlük hazırlayalım:

In [17]:
parameters_grid = [{'RF_model__n_estimators':[10, 20, 50],
                    'RF_model__max_features': ['sqrt', 'log2', 0.25, 0.5, 0.75],
                    'RF_model__max_depth' : [2, 4,5,6,7,8]}
                  ]

Gerekli tanımlamaları yaptıktan sonra artık çapraz doğrulamalı ızgara aramayı ayarlayabiliriz. Burada 10-katlı çapraz doğrulama (10-fold cross validation) kullanacağız ve performans ölçütü olarak doğruluk oranını (accuracy rate) tercih ediyoruz. 

In [18]:
search = GridSearchCV(estimator = pipe, param_grid = parameters_grid, cv = 10, scoring = 'accuracy', return_train_score=False, verbose=1, n_jobs=-1)

Artık elimizdeki eğitim verisini modellerimize uyum sağlatmaya (fitting) başlayabiliriz:

In [19]:
best_model = search.fit(X_train, y_train)

Fitting 10 folds for each of 90 candidates, totalling 900 fits


In [21]:
best_model.best_estimator_
# Pipeline(steps=[('preprocess',
#                  ColumnTransformer(n_jobs=-1,
#                                    transformers=[('num',
#                                                   Pipeline(steps=[('Imputer',
#                                                                    SimpleImputer()),
#                                                                   ('MinMaxScaler',
#                                                                    MinMaxScaler())]),
#                                                   Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
#        'Loan_Amount_Term', 'Credit_History'],
#       dtype='object')),
#                                                  ('cat',
#                                                   Pipeline(steps=[('Imputer',
#                                                                    SimpleImputer(fill_value='missing',
#                                                                                  strategy='constant')),
#                                                                   ('OneHotEncoder',
#                                                                    OneHotEncoder(handle_unknown='ignore'))]),
#                                                   Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
#        'Property_Area'],
#       dtype='object'))])),
#                 ('RF_model',
#                  RandomForestClassifier(class_weight='balanced', max_depth=2,
#                                         max_features=0.75, n_estimators=20,
#                                         n_jobs=-1))])

Pipeline(steps=[('preprocess',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num',
                                                  Pipeline(steps=[('Imputer',
                                                                   SimpleImputer()),
                                                                  ('MinMaxScaler',
                                                                   MinMaxScaler())]),
                                                  Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('Imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
           

In [22]:
ytrain_pred = best_model.predict(X_train)
ytest_pred = best_model.predict(X_test)

In [23]:
def TrainTestScores(y_train, y_train_pred, y_test, y_test_pred):
    
    scores = {"train_set": {"Accuracy" : accuracy_score(y_train, y_train_pred),
                            "Precision" : precision_score(y_train, y_train_pred),
                            "Recall" : recall_score(y_train, y_train_pred),                          
                            "F1 Score" : f1_score(y_train, y_train_pred),
                           "AUC": roc_auc_score(y_train, y_train_pred)},
    
              "test_set": {"Accuracy" : accuracy_score(y_test, y_test_pred),
                           "Precision" : precision_score(y_test, y_test_pred),
                           "Recall" : recall_score(y_test, y_test_pred),                          
                           "F1 Score" : f1_score(y_test, y_test_pred),
                          "AUC:": roc_auc_score(y_test, y_test_pred)}}
    
    return scores

In [24]:
TrainTestScores(y_train, ytrain_pred , y_test, ytest_pred)
# {'train_set': {'Accuracy': 0.8167006109979633,
#   'Precision': 0.8,
#   'Recall': 0.9824561403508771,
#   'F1 Score': 0.8818897637795275,
#   'AUC': 0.709348875544566},
#  'test_set': {'Accuracy': 0.7886178861788617,
#   'Precision': 0.7596153846153846,
#   'Recall': 0.9875,
#   'F1 Score': 0.8586956521739131,
#   'AUC:': 0.7030523255813954}}

{'train_set': {'Accuracy': 0.8167006109979633,
  'Precision': 0.8,
  'Recall': 0.9824561403508771,
  'F1 Score': 0.8818897637795275,
  'AUC': 0.709348875544566},
 'test_set': {'Accuracy': 0.7886178861788617,
  'Precision': 0.7596153846153846,
  'Recall': 0.9875,
  'F1 Score': 0.8586956521739131,
  'AUC:': 0.7030523255813954}}

In [25]:
pickle_out = open("classifier.pkl", mode = "wb") 
pickle.dump(best_model, pickle_out) 
pickle_out.close()